# Deployment of Emotion Classification Recommendation System for Mental Health Support

- This notebook is designed for deployment in a real-world application focused on emotion classification and provide a basic mental health support. It leverages a **pre-trained machine learning model (Stacking Classifier using the augmented dataset)** and **the OpenAI API** to classify emotions and provide personalized recommendations.
The deployment phase involves:
1. Loading the finalized models
2. Processing user input
3. Making accurate predictions
4. Offering actionable suggestions to support mental well-being

# Step 1: Google Drive links:
1. Stacking Model
2. TFIDF Vectorizer

**Stacking Model used for Emotion Classification link:**
- https://drive.google.com/file/d/1rgNxtmpNlTDEwbhoV2Lfibxlrd-bQ0K7/view?usp=sharing

**Tfidf vectorizer (pickle file) link:**
- https://drive.google.com/file/d/1au-rofToOgk8nzLnVoXhWFMNu9zKGfNr/view?usp=sharing

# Step 2: Import necessary libraries

In [ ]:
pip install openai==0.28

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from joblib import load
import openai

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 3: Load necessary models

In [ ]:
# Load the saved model
stacking_model = load('stacking_model_tuned.joblib')
print("Model loaded successfully.")

# Load the saved TF-IDF vectorizer
tfidf = load('tfidf_vectorizer_tuned.pkl')
print("TF-IDF vectorizer loaded successfully.")

Model loaded successfully.
TF-IDF vectorizer loaded successfully.


# Step 4: Preprocessing Function for the user input
- A custom preprocessing function converts input text to lowercase, removes special characters and stopwords, and applies stemming for standardization.

In [ ]:
# Preprocessing Function
def preprocess_text(input_text):
    # Convert to lowercase
    text = input_text.lower()

    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Remove stopwords
    stop = set(stopwords.words('english'))
    stop.difference_update(['in', 'on'])  # Keep 'in' and 'on' if needed
    text = " ".join(word for word in text.split() if word not in stop)

    # Apply stemming
    st = PorterStemmer()
    text = " ".join(st.stem(word) for word in text.split())

    return text

# Step 5: Function to make Predictions for the emotion
- The user's input is transformed using the TF-IDF vectorizer and passed to the stacking model to predict the emotion.

In [ ]:
# Function to Make Predictions
def predict_emotion(user_input):
    # Preprocess the input
    cleaned_text = preprocess_text(user_input)

    # Transform input using TF-IDF vectorizer
    text_vector = tfidf.transform([cleaned_text])

    # Predict using the loaded model
    prediction = stacking_model.predict(text_vector)

    # Output the prediction
    print(f"Predicted Emotion: {prediction[0]}")
    return prediction[0]

# Step 6: Setup my API Key
- The predicted emotion is used to generate mental health advice via the OpenAI API by constructing a supportive prompt for the model.

In [ ]:
# Set your API key
openai.api_key = "API_KEY"

# Function to generate mental health recommendations
def get_mental_health_recommendations(emotion):
    """
    Generate mental health recommendations based on user emotion.

    Args:
        emotion (str): The emotion provided by the user.

    Returns:
        str: ChatGPT's response with mental health recommendations.
    """
    # Define the prompt
    prompt = f"""
    A user has expressed feeling "{emotion}". As a mental health guide, provide recommendations or suggestions to help the user cope with this emotion in a supportive and compassionate manner.
    Focus on actionable and practical advice suitable for general use.
    """

    # Call the OpenAI ChatGPT API
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a mental health assistant."},
                {"role": "user", "content": prompt},
            ],
            temperature=0.7,
        )

        # Extract and return the response
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"An error occurred: {e}"


# Step 7: Main function to classify user's emotion and recommend resources about mental health
- The script prompts the user for their current emotion, processes it to classify the emotion, and returns practical mental health suggestions.

In [ ]:
# Main function
if __name__ == "__main__":
    # Prompt the user for their emotion
    user_emotion = input("How are you feeling right now? (e.g., sad, anxious, happy): ").strip()
    # Call the prediction function
    emotion_predicted = predict_emotion(user_emotion)

    # Get recommendations from ChatGPT
    recommendations = get_mental_health_recommendations(emotion_predicted)

    # Display the recommendations
    print("\nHere are some suggestions for you:")
    print(recommendations)

How are you feeling right now? (e.g., sad, anxious, happy): When I pass an examination which I did not think I did well.
Predicted Emotion: joy

Here are some suggestions for you:
1. Embrace the Joy: Allow yourself to fully experience and savor the feeling of joy. Acknowledge and appreciate the positive emotions you are feeling without judgment.

2. Share Your Joy: Share your joy with others by expressing gratitude or sending a kind message to someone you care about. Sharing your positive emotions can strengthen your relationships and spread positivity.

3. Practice Mindfulness: Engage in mindfulness techniques such as deep breathing, meditation, or grounding exercises to help you stay present and fully enjoy the moment of joy.

4. Capture the Moment: Consider journaling about the experience that brought you joy or capturing it through photos or artwork. Reflecting on positive moments can help you remember and revisit them during challenging times.

5. Engage in Self-Care: Use this mom